## Environment Preparation

* Load the SDK code from the local package directory
* Load the API key and secret in the .env file

In [1]:
# Quick hack to load local SDK code
import os

os.chdir(os.path.join(os.getcwd(), ".."))

In [2]:
# Load API key and secret from environment variables
from dotenv import load_dotenv
load_dotenv()

True

## ValidMind SDK Introduction

In [3]:
import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


In [4]:
# Initialize ValidMind SDK
import validmind as vm

vm.init(project="cl1jyvh2c000909lg1rk0a0zb")

True

In [5]:
df = pd.read_pickle("notebooks/datasets/_temp/df_loans_cleaned.pickle")

targets = vm.DatasetTargets(
    target_column="loan_status",
    class_labels={
        "Fully Paid": "Fully Paid",
        "Charged Off": "Charged Off",
    }
)

vm.log_dataset(df, "training", analyze=True, targets=targets)

True

In [6]:
results = vm.run_tests(df, target_column="loan_status", dataset_type="training", send=True)

Running data quality tests for "training" dataset...



100%|██████████| 7/7 [02:54<00:00, 24.92s/it]



Test suite has completed.
Sending results to ValidMind...
Successfully logged test results for test: class_imbalance
Successfully logged test results for test: duplicates
Successfully logged test results for test: cardinality
Successfully logged test results for test: missing
Successfully logged test results for test: pearson_correlation
Successfully logged test results for test: skewness
Successfully logged test results for test: zeros

Summary of results:

Test                 Passed      # Passed    # Errors    % Passed
-------------------  --------  ----------  ----------  ----------
class_imbalance      False              0           1           0
duplicates           False              0           1           0
cardinality          True              16           0         100
missing              False             23          55     29.4872
pearson_correlation  False              0          50           0
skewness             False             12          50     19.3548
zeros   

In [16]:

train_ds, val_ds = train_test_split(df, test_size=0.20)

x_train = train_ds.drop("loan_status", axis=1)
x_val = val_ds.drop("loan_status", axis=1)
y_train = train_ds.loc[:, "loan_status"].astype(str)
y_val = val_ds.loc[:, "loan_status"].astype(str)

In [9]:
xgb_model = xgb.XGBClassifier(early_stopping_rounds=10)
xgb_model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1], got ['Charged Off' 'Fully Paid']

In [8]:
y_pred = xgb_model.predict_proba(x_val)[:, -1]
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_val, predictions)

print(f"Accuracy: {accuracy}")

Accuracy: 0.848125


In [9]:
vm.log_model(xgb_model)

True